In [23]:
pip list

Package                      Version
---------------------------- -----------
aiohttp                      3.9.1
aiosignal                    1.3.1
altair                       4.2.2
annotated-types              0.6.0
asttokens                    2.2.1
async-timeout                4.0.3
attrs                        23.2.0
backcall                     0.2.0
blinker                      1.6.2
cachetools                   5.3.0
certifi                      2023.11.17
charset-normalizer           3.1.0
click                        8.1.7
click-plugins                1.1.1
cligj                        0.7.2
colorama                     0.4.6
comm                         0.1.4
contourpy                    1.0.7
cycler                       0.11.0
dataclasses-json             0.5.14
debugpy                      1.6.7.post1
decorator                    5.1.1
entrypoints                  0.4
executing                    1.2.0
faiss-cpu                    1.7.4
Fiona                        1.9.4.


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
import sys
sys.path.append('d:\\GitHub\\Mini-Projects\\LLM\\config')

from config import googleAPIKey
import re

with open('/GitHub/Mini-Projects/LLM/config/config.py', 'r') as databaseUrlFile:
    googleAPIKey = databaseUrlFile.read()

match = re.search(r'"([^"]*)"', googleAPIKey)

if match:
    googleAPIKey = match.group(1)
else:
    print("No match found")

In [20]:
from langchain.llms import GooglePalm

# temperature - control creativity level
llm = GooglePalm(google_api_key=googleAPIKey,temperature=0.7)

In [21]:
poem = llm("write a 4 line of my love for samosa")
print(poem)

**Samosas, oh samosas,**
**You are the food of my dreams.**
**Your crispy shell and savory filling**
**Fill me with joy and glee.**


In [24]:
essay = llm("write email requesting refund for electronic item")
print(essay)

```
Dear [Company Name],

I am writing to request a refund for the [product name] that I purchased on [date]. I am not satisfied with the product because [reason].

I have attached a copy of my receipt and a picture of the product. I have also included a description of the problem that I am having with the product.

I would appreciate it if you would process my refund as soon as possible. I would like to be refunded the full amount that I paid for the product.

Thank you for your time and consideration.

Sincerely,
[Your Name]
```
